In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import optuna

# Data preprocessing

In [2]:
# Charger le fichier
# file_path = "Augmented_df.csv"
# df = pd.read_csv(file_path, index_col=0)

file_path = "train.csv"
df = pd.read_csv(file_path)

# Afficher les premières lignes et les informations générales
df_info = df.info()
df_head = df.head()

df_info, df_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9415 entries, 0 to 9414
Columns: 4297 entries, smiles to class
dtypes: float64(4295), int64(1), object(1)
memory usage: 308.7+ MB


(None,
                                     smiles  BalabanJ     BertzCT       Chi0  \
 0    Brc1cc(Br)c(Oc2cc(Br)c(Br)cc2Br)cc1Br  2.415403  585.102114  14.154336   
 1      Brc1ccc2[nH]c3c(c2c1)CCCC3Nc1ccccc1  1.756595  776.517592  14.233840   
 2  Brc1ccc2c(NC3=NCC4(CN5CCC4CC5)O3)ncnn12  1.393870  796.361052  15.440947   
 3           Brc1cnc2nc(N3CCN4CCC3CC4)oc2c1  1.724265  605.709066  12.819626   
 4        Brc1cncc(COCC2(c3ccccc3)CCNCC2)c1  1.684628  597.045968  15.252866   
 
        Chi0n      Chi0v       Chi1     Chi1n     Chi1v     Chi2n  ...  \
 0   8.985436  18.501415   8.879918  4.851543  9.609532  3.495540  ...   
 1  11.695437  13.281434  10.309663  7.426852  8.219850  5.662312  ...   
 2  12.574323  14.160319  11.233913  7.945929  8.738927  6.525386  ...   
 3  10.549759  12.135755   9.292826  6.705489  7.498487  5.237927  ...   
 4  12.594869  14.180866  10.784424  7.719435  8.512433  5.900562  ...   
 
    fcfc_2039  fcfc_2040  fcfc_2041  fcfc_2042  fcfc_2043  fcfc_2

In [3]:
labels = df["class"]
labels.value_counts()

class
1    4816
0    4599
Name: count, dtype: int64

In [4]:
# df_filtered = df.drop(columns=[col for col in df.columns if col.startswith("fcfc") or col.startswith("ecfc")])
df_filtered = df.drop(columns=["class", "smiles"])
df_filtered.head()

,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,...,fcfc_2038,fcfc_2039,fcfc_2040,fcfc_2041,fcfc_2042,fcfc_2043,fcfc_2044,fcfc_2045,fcfc_2046,fcfc_2047
0,2.415403,585.102114,14.154336,8.985436,18.501415,8.879918,4.851543,9.609532,3.495540,8.621561,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.756595,776.517592,14.233840,11.695437,13.281434,10.309663,7.426852,8.219850,5.662312,6.577987,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.393870,796.361052,15.440947,12.574323,14.160319,11.233913,7.945929,8.738927,6.525386,7.337863,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.724265,605.709066,12.819626,10.549759,12.135755,9.292826,6.705489,7.498487,5.237927,6.153603,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.684628,597.045968,15.252866,12.594869,14.180866,10.784424,7.719435,8.512433,5.900562,6.816237,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# Supposons que df_features contient les features et df_target la colonne cible
X = df_filtered.copy()
y = labels.copy()

K-fold

In [6]:
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Model

In [7]:
model = XGBClassifier(n_estimators=250, learning_rate=0.16787519093314812, max_depth=7, subsample=0.8413149403890899, colsample_bytree=0.9206075463459951, random_state=42, eval_metric="auc")
# model = RandomForestClassifier()

In [8]:
scaler = StandardScaler()

In [9]:
scores = []

for train_index, test_index in kf.split(X):
        # Spliting des données
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        
        # **Entraînement du modèle**
        model.fit(X_train, y_train)
        
        # **Prédictions et évaluation**
        y_pred = model.predict(X_test)
        scores.append(accuracy_score(y_test, y_pred))

In [10]:
results = [{"Modèle:" : str(model), "Score moyen:" : np.mean(scores), "Écart-type:" : np.std(scores)}]
df_results = pd.DataFrame(results)

In [11]:
df_results

,Modèle:,Score moyen:,Écart-type:
0,"XGBClassifier(base_score=None, booster=None, c...",0.823048,0.005153
